In [1]:
import requests
import re

Muchos servidores web, al recibir múltiples peticiones en un corto tiempo de una misma IP, la bloquean para evitar saturaciones y problemas de servicio. Esto puede ser un problema para los scrapers ya que generan justamente este comportamiento.<br>
Para evitar ser detectados tendríamos que cambiar nuestra dirección IP pública antes de cada request, cosa que sería extremadamente lento y en muchos casos imposible, o podemos utilizar un **proxy**. Un proxy es un intermediario entre quien hace la petición (nuestro programa) y quien la recibe (el servidor) que nos permite enmascarar la IP de donde salió la request. Utilizando un proxy, el servidor web verá la IP de ese proxy y no la nuestra. Si bien no podemos elegir con qué dirección IP hacer la petición, sí podemos elegir a través de qué proxy hacerla.<br>
El sitio www.cualesmiip.com te permite ver cuál es la IP saliente de tu red. Si estás en una LAN, seguramente tu IP local sea algo como 192.18.x.x, pero la IP con la que salís al mundo, la IP de tu router asignada por tu ISP, será diferente.<br>

Armemos una función que averigue la IP que ve este sitio:

In [2]:
def get_my_ip(url='http://www.cualesmiip.com/', proxies=None):
    
###########################
# DESPUES ESTO
###########################    
    if proxies:
        if url.startswith('https'):
            if 'https' in proxies.keys():
                print('Utilizando proxy', proxies['https'])
            else:
                print('No hay proxy https configurado')
        elif url.startswith('http:'):
            if 'http' in proxies.keys():
                print('Utilizando proxy', proxies['http'])
            else:
                print('No hay proxy http configurado')            
        else:
            print('URL incorrecta')
            return None
    else:
        print('No hay proxy configurado')
###########################
# PRIMERO ESTO
###########################

    # Hacemos la request al sitio
    try:
        resp = requests.get(url, proxies=proxies)
    except Exception as e:
        print('Error haciendo la request.', e)
        return None

    # Verificamos el status_code
    if resp.status_code != 200:
        print('Error de status. Status code:', resp.status_code)
    
    # Compilamos la expresión regular con el patrón para la IP
    regex = re.compile(r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})')
    # Extraemos los strings que matcheen el patrón
    my_ip = regex.findall(resp.text)
    # y devolvemos el primero de ellos si es que lo hay
    return my_ip[0] if my_ip is not None else None

In [3]:
get_my_ip()

No hay proxy configurado


'181.47.77.129'

Esta es la IP que ve el sitio.<br>
Ahora utilicemos un PROXY para hacer la request. Debemos crear un diccionario e indicar la IP del proxy para cada protocolo. Podemos obtener una lista de proxies del sitio https://free-proxy-list.net/ o https://hidemy.name/es/proxy-list/

In [4]:
# Actualizar ip del proxy
proxy_dict = {'http':'http://23.237.23.73:3128',
              'https':'https://23.237.23.73:3128'}

Modifiquemos la función para que utilice e indique el proxy que está utilizando

In [5]:
get_my_ip(url='https://www.cualesmiip.com/', proxies=proxy_dict)

Utilizando proxy https://23.237.23.73:3128
Error haciendo la request. HTTPSConnectionPool(host='www.cualesmiip.com', port=443): Max retries exceeded with url: / (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fb3dba1b5f8>: Failed to establish a new connection: [Errno 110] Connection timed out',)))


Vemos que la IP que reconoce el sitio cambió y coincide con la IP del proxy que estamos usando.

Otro tipo de proxy que podemos utilizar es SOCKS. El proxy HTTP solamente funciona para ese protocolo, mientras que el proxy SOCKS funciona a más bajo nivel (TCP) y lo podemos utilizar para cualquier tipo de tráfico ya sean páginas web, programas, torrents, etc.

In [8]:
# Actualizar ip del proxy
proxy_dict_socks = {'http':'socks4://138.97.116.171:50659',
                    'https':'socks4://138.97.116.171:50659'}

In [9]:
get_my_ip(proxies=proxy_dict_socks)

Utilizando proxy socks4://138.97.116.171:50659


'138.97.116.190'